# Advanced Certification in AIML
## A Program by IIIT-H and TalentSprint

### Not for Grading

### Learning Objectives:

At the end of the experiment, you will be able to:
 
*  generate word embeddings using pre-trained models
*  visualize the similar words

### Setup Steps

In [ ]:
#@title Please enter your registration id to start: (e.g. P181900101) { run: "auto", display-mode: "form" }
Id = "" #@param {type:"string"}


In [ ]:
#@title Please enter your password (normally your phone number) to continue: { run: "auto", display-mode: "form" }
password = "" #@param {type:"string"}


In [ ]:
#@title Run this cell to complete the setup for this Notebook
from IPython import get_ipython

ipython = get_ipython()
  
notebook= "Demo_Word_Similarity" #name of the notebook
Answer = "Ungraded"
def setup():
    ipython.magic("sx wget https://cdn.iiith.talentsprint.com/aiml/Experiment_related_data/glove.6B.zip")
    ipython.magic("sx unzip glove.6B.zip")
    ipython.magic("sx wget https://cdn.talentsprint.com/talentsprint1/archives/sc/aiml/experiment_related_data/AIML_DS_GOOGLENEWS-VECTORS-NEGATIVE-300_STD.rar")
    ipython.magic("sx unrar e /content/AIML_DS_GOOGLENEWS-VECTORS-NEGATIVE-300_STD.rar")
    from IPython.display import HTML, display
    display(HTML('<script src="https://dashboard.talentsprint.com/aiml/record_ip.html?traineeId={0}&recordId={1}"></script>'.format(getId(),submission_id)))
    print("Setup completed successfully")
    return

def submit_notebook():
    
    ipython.magic("notebook -e "+ notebook + ".ipynb")
    
    import requests, json, base64, datetime

    url = "https://dashboard.talentsprint.com/xp/app/save_notebook_attempts"
    if not submission_id:
      data = {"id" : getId(), "notebook" : notebook, "mobile" : getPassword()}
      r = requests.post(url, data = data)
      r = json.loads(r.text)

      if r["status"] == "Success":
          return r["record_id"]
      elif "err" in r:        
        print(r["err"])
        return None        
      else:
        print ("Something is wrong, the notebook will not be submitted for grading")
        return None

    elif getAnswer() and getComplexity() and getAdditional() and getConcepts():
      f = open(notebook + ".ipynb", "rb")
      file_hash = base64.b64encode(f.read())

      data = {"complexity" : Complexity, "additional" :Additional, 
              "concepts" : Concepts, "record_id" : submission_id, 
              "answer" : Answer, "id" : Id, "file_hash" : file_hash,
              "feedback_experiments_input" : Comments, "notebook" : notebook}

      r = requests.post(url, data = data)
      r = json.loads(r.text)
      if "err" in r:        
        print(r["err"])
        return None   
      else:
        print("Your submission is successful.")
        print("Ref Id:", submission_id)
        print("Date of submission: ", r["date"])
        print("Time of submission: ", r["time"])
        print("View your submissions: https://aiml.iiith.talentsprint.com/notebook_submissions")
        # print("For any queries/discrepancies, please connect with mentors through the chat icon in LMS dashboard.")
      return submission_id
    else: submission_id
    

def getAdditional():
  try:
    if not Additional: 
      raise NameError
    else:
      return Additional  
  except NameError:
    print ("Please answer Additional Question")
    return None
def getComments():
  try:
    if not Comments:
      raise NameError
    else:
      return Comments
  except NameError:
    print ("Please answer Comments Question")
    return None

def getComplexity():
  try:
    if not Complexity:
      raise NameError
    else:
      return Complexity
  except NameError:
    print ("Please answer Complexity Question")
    return None
  
def getConcepts():
  try:
    if not Concepts:
      raise NameError
    else:
      return Concepts
  except NameError:
    print ("Please answer Concepts Question")
    return None

def getAnswer():
  try:
    if not Answer:
      raise NameError 
    else: 
      return Answer
  except NameError:
    print ("Please answer Question")
    return None

def getId():
  try: 
    return Id if Id else None
  except NameError:
    return None

def getPassword():
  try:
    return password if password else None
  except NameError:
    return None

submission_id = None
### Setup 
if getPassword() and getId():
  submission_id = submit_notebook()
  if submission_id:
    setup()
    
else:
  print ("Please complete Id and Password cells before running setup")


# PART I
#Find the similarity between words using GloVe

In [ ]:
# Importing required Packages
import pandas as pd
import numpy as np
import pprint
import warnings
warnings.filterwarnings("ignore")

* **Load the GloVe pretrained model**

  GloVe stands for “Global Vectors” for word representation. It is developed by Stanford for generating word embeddings. GloVe captures both global statistics and local statistics of a corpus, in order to come up with word vectors. 

In [ ]:
GloVe_Dict = {}

with open("glove.6B.50d.txt", 'r', encoding="utf-8") as f:
  for line in f:
      values = line.split()
      word = values[0]
      vector = np.asarray(values[1:], "float32")
      GloVe_Dict[word] = vector

* Develop GloVe Embeddings for the given list of words

In [ ]:
words = ['king','queen','river','water','ocean','tree','leaf','happy', 'glad', 'mother', 'daughter']

In [ ]:
pp = pprint.PrettyPrinter()

# Vector representation of a specific word 
print("Size of the vector is", len(GloVe_Dict["king"]))
pp.pprint(GloVe_Dict["king"])

In [ ]:
# Vector representation of each word using GloVe
vectors = []
for word in words:
  vector = GloVe_Dict[word]
  vectors.append(vector)
print("There are %d words and the vector size of each word is %d" %((len(vectors),len(vectors[0]))))

* Measure the similarity between the words using cosine_similarity


In [ ]:
# Importing the cosine similarity
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
word_similarity = []
for i,word_1 in enumerate(words):
  row_wise_simiarity = []
  for j,word_2 in enumerate(words):
    # Get the vectors of the word using GloVe
    vec_1, vec_2 = GloVe_Dict[word_1], GloVe_Dict[word_2]

    # As the vectors are in one dimensional, convert it to 2D by reshaping
    vec_1, vec_2 = np.array(vec_1).reshape(1,-1), np.array(vec_2).reshape(1,-1) 

    # Measure the cosine similarity between the vectors.
    similarity = cosine_similarity(vec_1,vec_2)
    row_wise_simiarity.append(np.array(similarity).item())

  # Store the cosine similarity values in a list  
  word_similarity.append(row_wise_simiarity)

# Create a DataFrame to view the similarity between words
pd.DataFrame(word_similarity, columns=words, index=words)


 *GloVe derives the semantic relationship between the words. Higher the cosine similarity, the words are relatively closer*

*For eg:* *The word 'King' is more closer to word 'Queen'*

* Visualize the words in 2D-plane by reducing the dimensions using PCA

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 2)

# All the vectors of each word was stored in vectors list. 
# Apply PCA and transform the data to 2 dimensions

reduced_vectors = pca.fit_transform(vectors)

from matplotlib import pyplot as plt
plt.figure(figsize=(7,5))
plt.scatter(reduced_vectors[:,0],reduced_vectors[:,1], s = 12, color = 'red')
plt.xlim([-3.5,4.5])
plt.ylim([-3.5,3.5])
x, y = reduced_vectors[:,0] , reduced_vectors[:,1]
for i in range(len(x)):
  plt.annotate(words[i],xy=(x[i], y[i]),xytext=(x[i]+0.05,y[i]+0.05))

# PART II
# Find the similarity between words using Word2Vec



* Load Gensim pretrained model

  * Gensim is an open source Python library for natural language processing. It is developed and is maintained by the Czech natural language processing researcher Radim Řehůřek and his company RaRe Technologies. 

  * Use gensim to load a word2vec model, pretrained on google news, covering approximately 3 million words and phrases. The vector size is 300 features.

  * Download the google news bin file with the limit 500000 words and save in a binary word2vec format. If **binary = True**, then the data will be saved in binary word2vec format, else it will be saved in plain text.

In [ ]:
import gensim

# Load Google news 300 vectors file
model = gensim.models.KeyedVectors.load_word2vec_format('AIML_DS_GOOGLENEWS-VECTORS-NEGATIVE-300_STD.bin', binary=True, limit=500000)

* Develop Word2Vec Embeddings for the list of words

In [ ]:
# Vector representation of a specific word 
pp = pprint.PrettyPrinter()
print("Size of the vector is", len(model["king"]))
pp.pprint(model["king"])

In [ ]:
# Vector representation of each word using Word2Vec
word2vec = []

for word in words:
  try:
    word2vec.append(model[word])
  except:
    pass
print("There are %d words and the vector size of each word is %d" %(len(word2vec),len(word2vec[0])))


* Measure the similarity between the words using cosine_similarity


In [ ]:
w2v_similarity = []
for i,word_1 in enumerate(words):
  w2v_row_wise_simiarity = []
  for j,word_2 in enumerate(words):
    # Get the vectors of the word using Word2Vec
    vec_1, vec_2 = model[word_1],model[word_2]

    # As the vectors are in one dimensional, convert it to 2D by reshaping
    vec_1, vec_2 = np.array(vec_1).reshape(1,-1), np.array(vec_2).reshape(1,-1) 

    # Measure the cosine similarity between two vectors
    similarity = cosine_similarity(vec_1,vec_2)
    w2v_row_wise_simiarity.append(np.array(similarity).item())

  # Store the cosine similarity values in a list    
  w2v_similarity.append(w2v_row_wise_simiarity)

pd.DataFrame(w2v_similarity, columns = words, index = words)


*Higher the cosine similarity, the words are more closer*

*For eg: The word 'King' is more similar to the word 'Queen'*

* Visualize the words in 2D-plane by reducing the dimensions using PCA

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 2)

# All the vectors of each word was stored in word2vec 
# Apply PCA and transform the data to 2 dimensions
reduced_w2v = pca.fit_transform(word2vec)

plt.figure(figsize=(8,5))
plt.scatter(reduced_w2v[:,0],reduced_w2v[:,1], s = 12, color = 'red')
plt.xlim([-2.5,2.5])
plt.ylim([-2.5,2.5])
x, y = reduced_w2v[:,0] , reduced_w2v[:,1]
for i in range(len(x)):
  plt.annotate(words[i],xy=(x[i], y[i]),xytext=(x[i]+0.05,y[i]+0.05))

## Please answer the questions below to complete the experiment:

In [ ]:
#@title How was the experiment? { run: "auto", form-width: "500px", display-mode: "form" }
Complexity = "" #@param ["","Too Simple, I am wasting time", "Good, But Not Challenging for me", "Good and Challenging for me", "Was Tough, but I did it", "Too Difficult for me"]


In [ ]:
#@title If it was too easy, what more would you have liked to be added? If it was very difficult, what would you have liked to have been removed? { run: "auto", display-mode: "form" }
Additional = "" #@param {type:"string"}


In [ ]:
#@title Can you identify the concepts from the lecture which this experiment covered? { run: "auto", vertical-output: true, display-mode: "form" }
Concepts = "" #@param ["","Yes", "No"]


In [ ]:
#@title  Text and image description/explanation and code comments within the experiment: { run: "auto", vertical-output: true, display-mode: "form" }
Comments = "" #@param ["","Very Useful", "Somewhat Useful", "Not Useful", "Didn't use"]


In [ ]:
#@title Run this cell to submit your notebook  { vertical-output: true }
try:
  if submission_id:
      return_id = submit_notebook()
      if return_id : submission_id =return_id
  else:
      print("Please complete the setup first.")
except NameError:
  print ("Please complete the setup first.")